# Semantic Kernel OpenAI Assistant Agent Code Interpreter

## Prepare File Paths of our files to be added to agent

In [1]:
import os

# Let's form the file paths that we will later pass to the assistant
csv_file_path_1 = os.path.join("Data/population/","PopulationByAdmin1.csv",)

csv_file_path_2 = os.path.join("Data/population/","PopulationByCountry.csv",)

## Step 1-2: Create an Agent and Thread

In [2]:
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.file_reference_content import FileReferenceContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

# Step 1: Create an assistant agent
agent = await AzureAssistantAgent.create(
        kernel=Kernel(),
        service_id="agent",
        name="SK_OpenAI_Assistant_Agent_Code_Interpreter",
        instructions="""
                Analyze the available data to provide an answer to the user's question.
                Always format response using markdown.
                Always include a numerical index that starts at 1 for any lists or tables.
                Always sort lists in ascending order.
                """,
        enable_code_interpreter=True,
        code_interpreter_filenames=[csv_file_path_1, csv_file_path_2],
)

# Step 2: Create a thread
thread_id = await agent.create_thread()

## Helper Methods
This will download any files created by the agent

In [3]:
async def download_file_content(agent, file_id: str):
    try:
        # Fetch the content of the file using the provided method
        response_content = await agent.client.files.content(file_id)

        # Get the current working directory of the file
        current_directory = "Data/skagents-output/"

        # Define the path to save the image in the current directory
        file_path = os.path.join(
            current_directory,  # Use the current directory of the file
            f"{file_id}.png",  # You can modify this to use the actual filename with proper extension
        )

        # Save content to a file asynchronously
        with open(file_path, "wb") as file:
            file.write(response_content.content)

        print(f"File saved to: {file_path}")
    except Exception as e:
        print(f"An error occurred while downloading file {file_id}: {str(e)}")


async def download_response_image(agent, file_ids: list[str]):
    if file_ids:
        # Iterate over file_ids and download each one
        for file_id in file_ids:
            await download_file_content(agent, file_id)

## Step 3-5: Helper Function
3. Add a message to the thread
4. Run the Assistant
5. Display the Assistant's Response

In [4]:
async def run_agent(user_question):
    # File IDs to store the references to the files
    file_ids: list[str] = []

    # STEP 3: Add a user question to the thread
    await agent.add_chat_message(
        thread_id=thread_id, 
        message=ChatMessageContent(role=AuthorRole.USER, content=user_question)
    )

    # STEP 4: Invoke the agent to get a response
    async for response in agent.invoke(thread_id=thread_id):
        # check if the response contains any code
        if response.metadata.get("code"):
            print("\n" + "Code from Agent:")
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)        
        else:
            #STEP 5: Print the Assistant response
            print(f"{response.content}", end="", flush=True)
                    
        print("\n**************")
        #Check if the response contains any file references
        file_ids.extend([
            item.file_id for item in response.items if isinstance(item, FileReferenceContent)
        ])

    # Download the file content
    await download_response_image(agent, file_ids)
    file_ids.clear()


In [ ]:
user_question = "What is the population of the country with the highest population?"
await run_agent(user_question)

## Appending Messages to the Thread

In [ ]:
user_question = "What is in third place?"
await run_agent(user_question)

In [ ]:
user_question = "Give me a column chart of the top 10 countries by population. Add values at the top of each column"
await run_agent(user_question)

In [ ]:
user_question = "Provide a column chart for countries whose names start with the same letter and sort the x axis by highest count to lowest (include all countries). Add values on top of each column"
await run_agent(user_question)

## Deleting Files, Thread, Agent

In [10]:
if agent is not None:
    [await agent.delete_file(file_id) for file_id in agent.code_interpreter_file_ids]
    await agent.delete_thread(thread_id)
    await agent.delete()